# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

This dataset contains California Housing Price data downloaded from https://raw.githubusercontent.com/ageron/handson-ml/master/ and was used as an example in Hands-On Machine Learning with Scikit-Learn & TensorFlow by Aurelien Geron. 

This dataset is an adapted version from the original data from the StatLib repository collected from the 1990 California census. 

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'California-housing-price-projection'
experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RSBG3LS77 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-137405
aml-quickstarts-137405
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
#create a compute target
amlcompute_cluster_name = "compute-housing"

# Verify that cluster does not exist already, if not create a new one with amlcompute_cluster_name
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [39]:
#check if the dataset exists and register the dataset if not
import train
found = False
key = "California_housing_price-1"
description_text = "California housing price from 1990 census"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key] 

if not found:       
        #If the dataset doesn't exist, then download data and register the Dataset in Workspace
#        DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
#        HOUSING_PATH = "datasets/housing"
#        HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"
#        df = train.fetch_housing_data(HOUSING_URL, HOUSING_PATH)
    path = r"https://github.com/second-husky/Capstone-Azure-ML/raw/master/starter_file/housing.csv"
    datastore = ws.get_default_datastore()
    dataset = Dataset.Tabular.from_delimited_files(path)
    dataset = dataset.register(workspace=ws,
                                name=key,
                                description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

KeyboardInterrupt: 

In [40]:
dataset = ws.datasets[key]

In [41]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "iterations" : 2,
    "primary_metric" : 'r2_score'
}
project_folder = "./housing"
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="median_house_value",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [62]:
exp_automl = Experiment(ws, 'AutoML-housing')
automl_run = exp_automl.submit(automl_config, show_output = False)
RUnDetails(automl_run).show()
automl_run.wait_for_completion()

Running on remote.


In [42]:
#create a pipeline and AutoMLStep
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [43]:
#create an AutoMLStep
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [44]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
# Use just-specified compute target ("cpu-cluster")
aml_run_config.target = compute_target

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk[automl]', 'azureml-dataprep[fuse,pandas]'], 
        pin_sdk_version=False)

In [45]:
%%writefile register_model.py
from azureml.core.model import Model, Dataset
from azureml.core.run import Run, _OfflineRun
from azureml.core import Workspace
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", required=True)
parser.add_argument("--model_path", required=True)
args = parser.parse_args()

print(f"model_name : {args.model_name}")
print(f"model_path: {args.model_path}")

run = Run.get_context()
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

model = Model.register(workspace=ws,
                       model_path=args.model_path,
                       model_name=args.model_name)

print("Registered version {0} of model {1}".format(model.version, model.name))

Overwriting register_model.py


In [47]:
from azureml.pipeline.core.graph import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

# The model name with which to register the trained model in the workspace.
model_name = PipelineParameter("model_name", default_value="California_housing_price")

register_step = PythonScriptStep(script_name="register_model.py",
                                       name="register_model",
                                       allow_reuse=False,
                                       arguments=["--model_name", model_name, "--model_path", model_data],
                                       inputs=[model_data],
                                       compute_target=compute_target,
                                       runconfig=aml_run_config)

In [48]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step, register_step])

In [49]:
# TODO: Submit your experiment
pipeline_run = experiment.submit(pipeline)

Created step automl_module [b34f4ce0][64f54584-9b36-49ba-8a47-60f7d4bcdac9], (This step will run and generate new outputs)
Created step register_model [f06990e6][a38e8fec-39e4-4c12-9a3c-5e85c7728989], (This step will run and generate new outputs)
Submitted PipelineRun 8fba800d-daf6-4885-aaef-ec79b4170679
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/California-housing-price-projection/runs/8fba800d-daf6-4885-aaef-ec79b4170679?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-137405/workspaces/quick-starts-ws-137405


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [50]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 8fba800d-daf6-4885-aaef-ec79b4170679
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/California-housing-price-projection/runs/8fba800d-daf6-4885-aaef-ec79b4170679?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-137405/workspaces/quick-starts-ws-137405
PipelineRun Status: Running


StepRunId: 9b54fd41-8805-4e93-a8e0-e1d07c62b944
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/California-housing-price-projection/runs/9b54fd41-8805-4e93-a8e0-e1d07c62b944?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-137405/workspaces/quick-starts-ws-137405
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '9b54fd41-8805-4e93-a8e0-e1d07c62b944', 'target': 'compute-housing', 'status': 'Completed', 'startTimeUtc': '2021-02-06T05:21:27.5047

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Response status code does not indicate success: 400 (Environment name can not start with the prefix AzureML. To alter a curated environment first create).\n{\n  \"error\": {\n    \"code\": \"UserError\",\n    \"severity\": null,\n    \"message\": \"Environment name can not start with the prefix AzureML. To alter a curated environment first create a copy of it.\",\n    \"messageFormat\": null,\n    \"messageParameters\": null,\n    \"referenceCode\": null,\n    \"detailsUri\": null,\n    \"target\": null,\n    \"details\": [],\n    \"innerError\": null,\n    \"debugInfo\": null\n  },\n  \"correlation\": {\n    \"operation\": \"075a2fd832ce344381506fdb6d09e48f\",\n    \"request\": \"ad1963533759ad43\"\n  },\n  \"environment\": \"southcentralus\",\n  \"location\": \"southcentralus\",\n  \"time\": \"2021-02-06T05:42:54.2254303+00:00\",\n  \"componen

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [29]:
#retrieve the metrics of all the child runs
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

ErrorResponseException: Unknown error

In [ ]:
#take a look at all the output metrics
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df_metrics = pd.DataFrame(deserialized_metrics_output)
df_metrics

In [ ]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

In [ ]:
print(best_model_output)

In [ ]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

In [ ]:
best_model

In [ ]:
type(best_model)

In [ ]:
#TODO: Save the best model
import joblib
os.makedirs('./outputs_automl', exist_ok = True)
ml_path = r'./outputs_automl/modelbest.joblib'
joblib.dump(value = best_model, filename=ml_path)

In [52]:
# Retrieved from Azure Machine Learning web UI
run_id = '8fba800d-daf6-4885-aaef-ec79b4170679'
experiment = ws.experiments['California-housing-price-projection']
run = next(run for run in experiment.get_runs() if run.id == run_id)

In [58]:
automl_run = next(r for r in run.get_children() if r.name == 'automl_module')
outputs = automl_run.get_outputs()
#best_autoML_run = automl_run.get_best_child()
outputs

{'metrics_data': <azureml.pipeline.core.run.StepRunOutput at 0x7f4a8e1cdb38>,
 'default_model_automl_module': <azureml.pipeline.core.run.StepRunOutput at 0x7f4a8e1cd7b8>,
 'default_metrics_automl_module': <azureml.pipeline.core.run.StepRunOutput at 0x7f4a8e1cd320>,
 'model_data': <azureml.pipeline.core.run.StepRunOutput at 0x7f4a8e188828>}

In [59]:
metrics = outputs['default_metrics_automl_module']
model = outputs['default_model_automl_module']

metrics.get_port_data_reference().download('.')
model.get_port_data_reference().download('.')

1

In [60]:
model

In [ ]:
model = Model.register(workspace=ws,
                       model_path=args.model_path,
                       model_name=args.model_name)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="California-Housing-Price", description="California Housing Price Prediction", version="1.0")

published_pipeline

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails
published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()